In [1]:
!pip install PyDrive hyperas hyperopt

In [2]:
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.datasets import mnist
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
def data():
    '''
    Data providing function:
    This function is separated from model() so that hyperopt
    won't reload data for each evaluation run.
    '''
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(60000, 784)
    X_test = X_test.reshape(10000, 784)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    nb_classes = 10
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    return X_train, Y_train, X_test, Y_test

In [0]:
def model(X_train, Y_train, X_test, Y_test):
    '''
    Model providing function:
    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    '''
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))

    # If we choose 'four', add an additional fourth layer
    if {{choice(['three', 'four'])}} == 'four':
        model.add(Dense(100))
        model.add({{choice([Dropout(0.5), Activation('linear')])}})
        model.add(Activation('relu'))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}},
                  metrics=['accuracy'])

    model.fit(X_train, Y_train,
              batch_size={{choice([64, 128])}},
              nb_epoch=1,
              verbose=2,
              validation_data=(X_test, Y_test))
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [5]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '17DLUohYnkWrnQg_NouLu4cK4LG0u--JH'})
download.GetContentFile('train_LbELtWX.zip')
!unzip train_LbELtWX.zip
# Copy/download the file
fid = drive.ListFile({'q':"title='Img_Class.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('Img_Class.ipynb')

Output hidden; open in https://colab.research.google.com to view.

In [18]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          max_evals=40,
                                          algo=tpe.suggest,
                                          notebook_name='Img_Class', # This is important!
                                          trials=Trials())

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    import os
except:
    pass

try:
    from pydrive.auth import GoogleAuth
except:
    pass

try:
    from pydrive.drive import GoogleDrive
except:
    pass

try:
    from google.colab import auth
except:
    pass

try:
    from oauth2client.client import GoogleCredentials
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dense': hp.choice('Dense', [256, 512, 1024]),
      

/content/temp_model.py:119: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.



Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 - 7s - loss: 1.6851 - acc: 0.4352 - val_loss: 0.7878 - val_acc: 0.8248

Test accuracy:
0.8248
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 - 11s - loss: 2.1364 - acc: 0.3013 - val_loss: 0.7163 - val_acc: 0.7927

Test accuracy:
0.7927
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 - 11s - loss: 1.9405 - acc: 0.3107 - val_loss: 0.6887 - val_acc: 0.8766

Test accuracy:
0.8766
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 - 7s - loss: 0.7151 - acc: 0.7721 - val_loss: 0.2099 - val_acc: 0.9364

Test accuracy:
0.9364
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 - 7s - loss: 0.5157 - acc: 0.8427 - val_loss: 0.1414 - val_acc: 0.9581

Test accuracy:
0.9581
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 - 6s - loss: 2.7188 - acc: 0.1103 - val_loss: 2.1607 - val_acc: 0.4118

Test accuracy:
0.4118
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 - 7s - 

In [19]:
best_run

{'Activation': 0,
 'Dense': 2,
 'Dropout': 0.10342491821973374,
 'Dropout_1': 0.37352344582529806,
 'Dropout_2': 0,
 'add': 0,
 'batch_size': 0,
 'optimizer': 1}